In [1]:
import re
import requests
from pathlib import Path
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.parse import urljoin
import mimetypes

In [2]:
!rm -rf mainframe/static/
!mkdir -p mainframe/static/mainframe/plugins/
!cp -ar AdminLTE-3.2.0/plugins/fontawesome-free  mainframe/static/mainframe/plugins/
!cp -ar AdminLTE-3.2.0/plugins//bootstrap  mainframe/static/mainframe/plugins/

In [3]:
admin_lte_home = Path('AdminLTE-3.2.0')
pages_home = admin_lte_home.joinpath('dist/pages')
app_home = Path('mainframe')
template_home = app_home.joinpath('templates/mainframe')
static_home = app_home.joinpath('static/mainframe')

In [4]:
template_home.mkdir(exist_ok=True, parents=True)
static_home.mkdir(exist_ok=True, parents=True)

In [5]:
def extract_resource(url):
    """ """
    path = admin_lte_home.joinpath(url)
    if path.exists(): 
        new_path = static_home.joinpath(url.replace('../../', ''))
        new_path.parent.mkdir(exist_ok=True, parents=True)
        new_path.write_bytes(path.read_bytes())        
    else:
        new_path = static_home.joinpath(re.sub('https://.*/npm/@*', '', url).replace('@', '/'))
        new_path.parent.mkdir(exist_ok=True, parents=True)
        new_path.write_bytes(requests.get(url).content)
        mime_type, _ = mimetypes.guess_type(url)         
    return new_path

In [6]:
def solve_resource(html):
    """ """
    soup = BeautifulSoup(html)    
    attrname_dict = {'link': 'href', 'script': 'src', 'img': 'src'}
    for tag in soup.find_all(['script', 'link', 'img']):
        attrname = attrname_dict.get(tag.name, None)
        if not attrname: continue
        url = tag.get(attrname)
        if not url: continue
        # print(url)
        new_path = extract_resource(url)
        tag[attrname] = Path('/dc').joinpath(new_path.relative_to(app_home)).as_posix()
    return soup.prettify()

In [7]:
original_html = solve_resource( admin_lte_home.joinpath('iframe.html').read_text())

In [8]:
soup = BeautifulSoup(original_html, 'html.parser')

# Remove demo script
[script.replace_with('') for script in soup.select('script[src*="demo"]')]

# Clear all comments
for commnet in soup.find_all(string=lambda text: isinstance(text, Comment)):
    commnet.replace_with('')

# Split pages
qs = 'body div[class="wrapper"]'
for tag in [x for x in soup.select(qs)[0].children if str(x).strip()][:-1]:
    tag_name = 'iframe' if tag.name == 'div' else tag.name
    path = template_home.joinpath(f'{ tag_name }.html')
    path.write_text(tag.prettify())    
    tag.replace_with(f'{{% include "{ app_home }/{ tag_name }.html" %}}')

# Clear menu
aside = BeautifulSoup(template_home.joinpath('aside.html').read_text())

# nav_item
nav_item = [_ for _ in aside.select('aside .mt-2 ul li') if not _.select('li')][0]
nav_item.select('a')[0].attrs['href'] = '{{ href }}'
nav_item.select('p')[0].string = '{{ name }}'
template_home.joinpath('nav_item.html').write_text(
    '{% load util %}\n' + nav_item.prettify())

# nav_treeview
treeview_content = BeautifulSoup('''
{% for key, value in data.items %}
    {% if value|get_type == "str" %}
        {% include "mainframe/nav_item.html" with name=key href=value %}
    {% elif  value|get_type == "dict" %}
        {% include "mainframe/nav_treeview.html" with name=key data=value %}
    {% else %}
         <div>Unknown Menu Type: {{ value|get_type }}</div>
    {% endif %} 
{% endfor %}
''')
aside = BeautifulSoup(template_home.joinpath('aside.html').read_text())
nav_treeview = [_ for _ in aside.select('aside .mt-2 ul li') if _.select('li')][0]
[_.replace_with('') for _ in nav_treeview.select('ul li')]
p_content = BeautifulSoup(f"{{{{ name }}}}{ nav_treeview.select('p i')[0] }")
nav_treeview.find('p').string = ''
nav_treeview.find('p').append(p_content)
nav_treeview.find('ul').string = ''
nav_treeview.find('ul').append(treeview_content)
template_home.joinpath('nav_treeview.html').write_text(
    '{% load util %}\n' + nav_treeview.prettify())

aside.select('aside .mt-2 ul')[0].string = f'{{% include "{ app_home }/menu.html" %}}'
template_home.joinpath('aside.html').write_text(aside.prettify())
template_home.joinpath('index.html').write_text(soup.prettify())

1516

In [9]:
print(nav_item.prettify())

<li class="nav-item">
 <a class="nav-link" href="{{ href }}">
  <i class="far fa-circle nav-icon">
  </i>
  <p>
   {{ name }}
  </p>
 </a>
</li>



In [10]:
nav_treeview.p

<p>{{ name }}<i class="right fas fa-angle-left">
</i></p>